In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime as dt  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import pandas as pd

# Import the backtrader platform
import backtrader as bt

# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        if self.dataclose[0] < self.dataclose[-1]:
            # current close less than previous close

            if self.dataclose[-1] < self.dataclose[-2]:
                # previous close less than the previous close

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.buy()




In [2]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    cerebro.addstrategy(TestStrategy)

    tickers_list = ['AAPL']
    df_tic = pd.read_hdf("datasets/df_SnP_500_ohlcv.h5", "df", mode = 'r')
    df_tic = df_tic[df_tic['tic'].isin(tickers_list)]

    df_tic = df_tic.set_index('date')
    data = bt.feeds.PandasData(dataname = df_tic,
                                            datetime=None, 
                                            open =1,
                                            high=2,
                                            low=3,
                                            close=4,
                                            volume=6,
                                            openinterest=-1,
                                            timeframe = bt.TimeFrame.Days,
                                            fromdate=dt.datetime(2023, 1, 1),  # Specify the start date
                                            todate=dt.datetime(2023, 6, 30),   # Specify the end date
                                        )


    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2023-01-03, Close, 125.07
2023-01-04, Close, 126.36
2023-01-05, Close, 125.02
2023-01-06, Close, 129.62
2023-01-09, Close, 130.15
2023-01-10, Close, 130.73
2023-01-11, Close, 133.49
2023-01-12, Close, 133.41
2023-01-13, Close, 134.76
2023-01-17, Close, 135.94
2023-01-18, Close, 135.21
2023-01-19, Close, 135.27
2023-01-20, Close, 137.87
2023-01-23, Close, 141.11
2023-01-24, Close, 142.53
2023-01-25, Close, 141.86
2023-01-26, Close, 143.96
2023-01-27, Close, 145.93
2023-01-30, Close, 143.00
2023-01-31, Close, 144.29
2023-02-01, Close, 145.43
2023-02-02, Close, 150.82
2023-02-03, Close, 154.50
2023-02-06, Close, 151.73
2023-02-07, Close, 154.65
2023-02-08, Close, 151.92
2023-02-09, Close, 150.87
2023-02-09, BUY CREATE, 150.87
2023-02-10, Close, 151.01
2023-02-13, Close, 153.85
2023-02-14, Close, 153.20
2023-02-15, Close, 155.33
2023-02-16, Close, 153.71
2023-02-17, Close, 152.55
2023-02-17, BUY CREATE, 152.55
2023-02-21, Close, 148.48
2023-02-21, BUY CR

In previous example, several “BUY” creation orders were issued. A couple of important things are clearly missing.

- The order was created but it is unknown if it was executed, when and at what price.
   - The next example will build upon that by listening to notifications of order status.

- How many shares are being bought, what asset is being bought and how are orders being executed.   
   - The stake is provided behind the scenes by a position sizer which uses a fixed stake, being the default “1”.

- The order is executed “At Market”. The broker (shown in previous examples) executes this using the opening price of the next bar, because that’s the 1st tick after the current under examination bar.

- The order is executed so far without any commission (more on that later)

- Do not only buy … but SELL
   - After knowing how to enter the market (long), an “exit concept” is needed and also understanding whether the strategy is in the market.

   - Luckily a Strategy object offers access to a position attribute for the default data feed

Methods buy and sell return the created (not yet executed) order. Changes in orders’ status will be notified to the strategy via a notify method

The “exit concept” will be an easy one:
- Exit after 5 bars (on the 6th bar) have elapsed for good or for worse
- Please notice that there is no “time” or “timeframe” implied: number of bars. The bars can represent 1 minute, 1 hour, 1 day, 1 week or any other time period.
- Although we know the data source is a daily one, the strategy makes no assumption about that.



In [3]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 5):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [4]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    tickers_list = ['AAPL']
    df_tic = pd.read_hdf("datasets/df_SnP_500_ohlcv.h5", "df", mode = 'r')
    df_tic = df_tic[df_tic['tic'].isin(tickers_list)]

    df_tic = df_tic.set_index('date')
    data = bt.feeds.PandasData(dataname = df_tic,
                                            datetime=None, 
                                            open =1,
                                            high=2,
                                            low=3,
                                            close=4,
                                            volume=6,
                                            openinterest=-1,
                                            timeframe = bt.TimeFrame.Days,
                                            fromdate=dt.datetime(2023, 1, 1),  # Specify the start date
                                            todate=dt.datetime(2023, 8, 30),   # Specify the end date
                                        )

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2023-01-03, Close, 125.07
2023-01-04, Close, 126.36
2023-01-05, Close, 125.02
2023-01-06, Close, 129.62
2023-01-09, Close, 130.15
2023-01-10, Close, 130.73
2023-01-11, Close, 133.49
2023-01-12, Close, 133.41
2023-01-13, Close, 134.76
2023-01-17, Close, 135.94
2023-01-18, Close, 135.21
2023-01-19, Close, 135.27
2023-01-20, Close, 137.87
2023-01-23, Close, 141.11
2023-01-24, Close, 142.53
2023-01-25, Close, 141.86
2023-01-26, Close, 143.96
2023-01-27, Close, 145.93
2023-01-30, Close, 143.00
2023-01-31, Close, 144.29
2023-02-01, Close, 145.43
2023-02-02, Close, 150.82
2023-02-03, Close, 154.50
2023-02-06, Close, 151.73
2023-02-07, Close, 154.65
2023-02-08, Close, 151.92
2023-02-09, Close, 150.87
2023-02-09, BUY CREATE, 150.87
2023-02-10, BUY EXECUTED, 149.46
2023-02-10, Close, 151.01
2023-02-13, Close, 153.85
2023-02-14, Close, 153.20
2023-02-15, Close, 155.33
2023-02-16, Close, 153.71
2023-02-17, Close, 152.55
2023-02-17, SELL CREATE, 152.55
2023-02-21

## Adding Commissions

Let’s add a reasonable 0.1% commision rate per operation (both for buying and selling … yes the broker is avid …)

A single line will suffice for it:

In [7]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 5):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [8]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    tickers_list = ['AAPL']
    df_tic = pd.read_hdf("datasets/df_SnP_500_ohlcv.h5", "df", mode = 'r')
    df_tic = df_tic[df_tic['tic'].isin(tickers_list)]

    df_tic = df_tic.set_index('date')
    data = bt.feeds.PandasData(dataname = df_tic,
                                            datetime=None, 
                                            open =1,
                                            high=2,
                                            low=3,
                                            close=4,
                                            volume=6,
                                            openinterest=-1,
                                            timeframe = bt.TimeFrame.Days,
                                            fromdate=dt.datetime(2023, 1, 1),  # Specify the start date
                                            todate=dt.datetime(2023, 8, 30),   # Specify the end date
                                        )

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)
    
    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.001)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2023-01-03, Close, 125.07
2023-01-04, Close, 126.36
2023-01-05, Close, 125.02
2023-01-06, Close, 129.62
2023-01-09, Close, 130.15
2023-01-10, Close, 130.73
2023-01-11, Close, 133.49
2023-01-12, Close, 133.41
2023-01-13, Close, 134.76
2023-01-17, Close, 135.94
2023-01-18, Close, 135.21
2023-01-19, Close, 135.27
2023-01-20, Close, 137.87
2023-01-23, Close, 141.11
2023-01-24, Close, 142.53
2023-01-25, Close, 141.86
2023-01-26, Close, 143.96
2023-01-27, Close, 145.93
2023-01-30, Close, 143.00
2023-01-31, Close, 144.29
2023-02-01, Close, 145.43
2023-02-02, Close, 150.82
2023-02-03, Close, 154.50
2023-02-06, Close, 151.73
2023-02-07, Close, 154.65
2023-02-08, Close, 151.92
2023-02-09, Close, 150.87
2023-02-09, BUY CREATE, 150.87
2023-02-10, BUY EXECUTED, Price: 149.46, Cost: 149.46, Comm 0.15
2023-02-10, Close, 151.01
2023-02-13, Close, 153.85
2023-02-14, Close, 153.20
2023-02-15, Close, 155.33
2023-02-16, Close, 153.71
2023-02-17, Close, 152.55
2023-02-17

The “NET” profit is already cash in the bag. There may be a change that NET profit will not match with profit in Final Portfolio Value. Unfortunately (or fortunately to better understand the platform) there is a chance that an open position on the last day of the Data Feed. Even if a SELL operation has been sent … IT HAS NOT YET BEEN EXECUTED.

### Customizing the Strategy: Parameters

It would a bit unpractical to hardcode some of the values in the strategy and have no chance to change them easily. Parameters come in handy to help.

Definition of parameters is easy and looks like:

params = (('myparam', 27), ('exitbars', 5),)

With either formatting parametrization of the strategy is allowed when adding the strategy to the Cerebro engine:

Add a strategy

cerebro.addstrategy(TestStrategy, myparam=20, exitbars=7)

In [15]:
def read_data(symbol = 'AAPL'):
    tickers_list = [symbol]
    df_tic = pd.read_hdf("datasets/df_SnP_500_ohlcv.h5", "df", mode = 'r')
    df_tic = df_tic[df_tic['tic'].isin(tickers_list)]

    df_tic = df_tic.set_index('date')
    data = bt.feeds.PandasData(dataname = df_tic,
                                            datetime=None, 
                                            open =1,
                                            high=2,
                                            low=3,
                                            close=4,
                                            volume=6,
                                            openinterest=-1,
                                            timeframe = bt.TimeFrame.Days,
                                            fromdate=dt.datetime(2023, 1, 1),  # Specify the start date
                                            todate=dt.datetime(2023, 8, 30),   # Specify the end date
                                        )
    return data

In [16]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('exitbars', 5),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + self.params.exitbars):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [17]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    data = read_data(symbol = 'AAPL')

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)

    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.001)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2023-01-03, Close, 125.07
2023-01-04, Close, 126.36
2023-01-05, Close, 125.02
2023-01-06, Close, 129.62
2023-01-09, Close, 130.15
2023-01-10, Close, 130.73
2023-01-11, Close, 133.49
2023-01-12, Close, 133.41
2023-01-13, Close, 134.76
2023-01-17, Close, 135.94
2023-01-18, Close, 135.21
2023-01-19, Close, 135.27
2023-01-20, Close, 137.87
2023-01-23, Close, 141.11
2023-01-24, Close, 142.53
2023-01-25, Close, 141.86
2023-01-26, Close, 143.96
2023-01-27, Close, 145.93
2023-01-30, Close, 143.00
2023-01-31, Close, 144.29
2023-02-01, Close, 145.43
2023-02-02, Close, 150.82
2023-02-03, Close, 154.50
2023-02-06, Close, 151.73
2023-02-07, Close, 154.65
2023-02-08, Close, 151.92
2023-02-09, Close, 150.87
2023-02-09, BUY CREATE, 150.87
2023-02-10, BUY EXECUTED, Price: 149.46, Cost: 1494.60, Comm 1.49
2023-02-10, Close, 151.01
2023-02-13, Close, 153.85
2023-02-14, Close, 153.20
2023-02-15, Close, 155.33
2023-02-16, Close, 153.71
2023-02-17, Close, 152.55
2023-02-1